# Work in progress

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# My imports
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_root = "../input"
output_dir = './'

In [ ]:
test = pd.read_csv(os.path.join(data_root, "test.csv"))
test.head()

In [ ]:
train = pd.read_csv(os.path.join(data_root, "train.csv"))
train.head()

In [ ]:
merchants = pd.read_csv(os.path.join(data_root, "merchants.csv"))
merchants.head()

In [ ]:
historical_transactions = pd.read_csv(os.path.join(data_root, "historical_transactions.csv"))
historical_transactions.head()

In [ ]:
new_merchant_transactions = pd.read_csv(os.path.join(data_root, "new_merchant_transactions.csv"))
new_merchant_transactions.head()

In [ ]:
engineered_test = pd.DataFrame()

In [ ]:
engineered_test = test

In [ ]:
engineered_train = pd.DataFrame()
engineered_train_outlier = pd.DataFrame()

# Take a look at the so called training features

In [ ]:
plt.figure()
train['target'].hist()
plt.show()

In [ ]:
train[train['target'] < -20]['target'].value_counts()

Huh? Is this some kind of place holder for spam cards??
Maybe a binary classifier should be used to determine this group

In [ ]:
engineered_train = train[train['target'] > -20]
engineered_train.head()

In [ ]:
engineered_train_outlier = train[train['target'] < -20]
engineered_train_outlier.head()

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(24,4))
engineered_train['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='train feature_1')
engineered_train['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='train feature_2')
engineered_train['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='train feature_3')
test['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[3], color='teal', title='test feature_1')
test['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[4], color='brown', title='test feature_2')
test['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[5], color='gold', title='test feature_3')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(24,4))
for i in range(5):
    title = "feature_1=" + str(i+1)
    engineered_train[engineered_train['feature_1'] == i+1]['target'].hist(ax=ax[i], bins=50)
    ax[i].set_title(title)
engineered_train['target'].hist(ax=ax[-1], bins=50)
ax[-1].set_title('overall train target')
fig.suptitle('Target distribution with all feature_1 values')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20,4))
for i in range(3):
    title = "feature_2=" + str(i+1)
    engineered_train[engineered_train['feature_2'] == i+1]['target'].hist(ax=ax[i], bins=50)
    ax[i].set_title(title)
engineered_train['target'].hist(ax=ax[-1], bins=50)
ax[-1].set_title('overall train target')
fig.suptitle('Target distribution with all feature_2 values')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,4))
for i in range(2):
    title = "feature_3=" + str(i)
    engineered_train[engineered_train['feature_3'] == i]['target'].hist(ax=ax[i], bins=50)
    ax[i].set_title(title)
engineered_train['target'].hist(ax=ax[-1], bins=50)
ax[-1].set_title('overall train target')
fig.suptitle('Target distribution with all feature_3 values')
plt.show()

It seems that feature 1, 2, and 3 are pretty much useless

Then how about active month?

In [ ]:
def month_to_timestamp(month):
    time_format = '%Y-%m'
    if type(month) is str:
        timestamp = datetime.strptime(month,time_format).timestamp()
        return timestamp
    else:
        return 1.5 * 1000000000

In [ ]:
engineered_train['first_active_month_timestamp'] = train[train['target'] > -20]['first_active_month'].apply(month_to_timestamp)

In [ ]:
engineered_train_outlier['first_active_month_timestamp'] = train[train['target'] < -20]['first_active_month'].apply(month_to_timestamp)

In [ ]:
engineered_test['first_active_month_timestamp'] = test['first_active_month'].apply(month_to_timestamp)

In [ ]:
engineered_train.head()

In [ ]:
engineered_train_outlier.head()

In [ ]:
engineered_test.head()

In [ ]:
plt.figure()
engineered_train['first_active_month_timestamp'].value_counts().sort_index().plot(label='train')
engineered_test['first_active_month_timestamp'].value_counts().sort_index().plot(label='test')
plt.legend()
plt.show()

In [ ]:
plt.figure()
engineered_train.groupby('first_active_month_timestamp')['target'].mean().plot()
plt.title('mean target given timestamp')
plt.ylabel('mean target')
plt.show()

In [ ]:
plt.figure()
engineered_train.groupby('first_active_month_timestamp')['target'].var().plot()
plt.title('var target given timestamp')
plt.ylabel('var target')
plt.show()

It seems that time indicates both the mean and variance of the target value

# Now what does new_merchant_transactions give us?

In [ ]:
new_merchant_transactions.head()

In [ ]:
city_counts = new_merchant_transactions['city_id'].value_counts()
unique_city = len(city_counts)
max_city = city_counts.idxmax()
max_count = city_counts[max_city]
max_percentage = max_count / len(new_merchant_transactions)
print('unique city: ', unique_city)
print('majority city: ', max_city)
print('count: ', max_count)
print('percentage: ', max_percentage)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
train.merge(new_merchant_transactions[new_merchant_transactions['city_id'] == 69].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=40, ax=ax[0])
train.merge(new_merchant_transactions[new_merchant_transactions['city_id'] != 69].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=40, ax=ax[1])
ax[0].set_title('with city_id as 69')
ax[1].set_title('with city_id as not 69')
plt.show()

In [ ]:
plt.figure(figsize=(20,4))
new_merchant_transactions['subsector_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of subsector_id in new_merchant_transactions')
plt.xlabel('subsector_id')
plt.ylabel('count')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))
train.merge(new_merchant_transactions.groupby('card_id')['subsector_id'].nunique().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='subsector_id', ax=ax[0])
engineered_train.merge(new_merchant_transactions.groupby('card_id')['subsector_id'].nunique().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='subsector_id', ax=ax[1])
ax[0].set_title('for all the training data')
ax[1].set_title('for training data without outliers')
fig.suptitle('target and subsector_id coverage')
plt.show()

Hmmmmm, but is this just because it is a indication of more purchase?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))
train.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[0])
engineered_train.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[1])
ax[0].set_title('for all the training data')
ax[1].set_title('for training data without outliers')
fig.suptitle('target and mean purchase amount')
plt.show()

In [ ]:
engineered_train['new_purchase_mean'] = engineered_train.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='left', on='card_id')['purchase_amount']

In [ ]:
engineered_test['new_purchase_mean'] = engineered_test.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='left', on='card_id')['purchase_amount']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))
train.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[0])
engineered_train.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[1])
ax[0].set_title('for all the training data')
ax[1].set_title('for training data without outliers')
fig.suptitle('target and total purchase amount')
plt.show()

In [ ]:
engineered_train['new_purchase_sum'] = engineered_train.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='left', on='card_id')['purchase_amount'].fillna(0)

In [ ]:
engineered_test['new_purchase_sum'] = engineered_test.merge(new_merchant_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='left', on='card_id')['purchase_amount'].fillna(0)

In [ ]:
plt.figure()
new_merchant_transactions['category_3'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of category_3 in new_merchant_transactions')
plt.xlabel('category_3')
plt.ylabel('count')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))
train.merge(new_merchant_transactions[new_merchant_transactions['category_3'] == 'A'].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[0])
train.merge(new_merchant_transactions[new_merchant_transactions['category_3'] == 'B'].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[1])
train.merge(new_merchant_transactions[new_merchant_transactions['category_3'] == 'C'].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[2])
ax[0].set_title('with category_2 as A')
ax[1].set_title('with category_2 as B')
ax[2].set_title('with category_2 as C')
plt.show()

In [ ]:
plt.figure()
new_merchant_transactions['category_2'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of category_2 in new_merchant_transactions')
plt.xlabel('category_2')
plt.ylabel('count')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15,3))
train.merge(new_merchant_transactions[new_merchant_transactions['category_2'] == 1].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[0])
train.merge(new_merchant_transactions[new_merchant_transactions['category_2'] == 2].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[1])
train.merge(new_merchant_transactions[new_merchant_transactions['category_2'] == 3].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[2])
train.merge(new_merchant_transactions[new_merchant_transactions['category_2'] == 4].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[3])
train.merge(new_merchant_transactions[new_merchant_transactions['category_2'] == 5].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[4])
ax[0].set_title('with category_2 as 1')
ax[1].set_title('with category_2 as 2')
ax[2].set_title('with category_2 as 3')
ax[3].set_title('with category_2 as 4')
ax[4].set_title('with category_2 as 5')
plt.show()

In [ ]:
plt.figure()
new_merchant_transactions['category_1'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of category_1 in new_merchant_transactions')
plt.xlabel('category_1')
plt.ylabel('count')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
train.merge(new_merchant_transactions[new_merchant_transactions['category_1'] == 'N'].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[0])
train.merge(new_merchant_transactions[new_merchant_transactions['category_1'] == 'Y'].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[1])
ax[0].set_title('with category_1 as Y')
ax[1].set_title('with category_1 as N')
plt.show()

In [ ]:
plt.figure()
new_merchant_transactions['installments'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of installments in new_merchant_transactions')
plt.xlabel('installments')
plt.ylabel('count')
plt.show()

1. Huh? 999 and -1? What is that exactly?

In [ ]:
new_merchant_transactions[(new_merchant_transactions['installments'] == -1) | (new_merchant_transactions['installments'] == 999)]['installments'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
train.merge(new_merchant_transactions[new_merchant_transactions['installments'] == -1].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[0])
train.merge(new_merchant_transactions[new_merchant_transactions['installments'] != -1].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[1])
ax[0].set_title('with installments as -1')
ax[1].set_title('with installments as not -1')
plt.show()

So -1 might just mean it is missing. It doesn't indicate anything. For 999, there are too few in new_merchant_transactions. Will have to find out in historical one

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
train.merge(new_merchant_transactions[(new_merchant_transactions['installments'] == 0) | (new_merchant_transactions['installments'] == 1)].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[0])
train.merge(new_merchant_transactions[(new_merchant_transactions['installments'] != 0) & (new_merchant_transactions['installments'] != 1)].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[1])
ax[0].set_title('with installments as 0 or 1')
ax[1].set_title('with installments as otherwise')
plt.show()

Well... very likely it is another useless feature

In [ ]:
new_merchant_transactions['authorized_flag'].value_counts()

Wat? really? Maybe only historial data make some differentiation in this field?

In [ ]:
plt.figure()
new_merchant_transactions['state_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of state_id in new_merchant_transactions')
plt.xlabel('state_id')
plt.ylabel('count')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
train.merge(new_merchant_transactions[new_merchant_transactions['state_id'] == 9].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[0])
train.merge(new_merchant_transactions[new_merchant_transactions['state_id'] != 9].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=20, ax=ax[1])
ax[0].set_title('with state_id as 9')
ax[1].set_title('with state_id as not 9')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
engineered_train.merge(new_merchant_transactions[new_merchant_transactions['state_id'] == 9].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=40, ax=ax[0])
engineered_train.merge(new_merchant_transactions[new_merchant_transactions['state_id'] != 9].groupby('card_id')['installments'].mean().to_frame(), how='inner', on='card_id')['target'].hist(bins=40, ax=ax[1])
ax[0].set_title('with state_id as 9')
ax[1].set_title('with state_id as not 9')
plt.show()

In [ ]:
unique_in_new_merchant_transactions = len(new_merchant_transactions['card_id'].value_counts())
unique_in_train = len(engineered_train['card_id'].value_counts())
common_in_two = len(pd.merge(engineered_train, new_merchant_transactions.groupby('card_id')['month_lag'].mean().to_frame(), on='card_id', how='inner'))
print('unique card ids in new_merchant_transactions: ', unique_in_new_merchant_transactions)
print('unique card ids in train: ', unique_in_train)
print('common values: ', common_in_two)
print('missing values: ', unique_in_train - common_in_two)

In [ ]:
mean_month_lag = new_merchant_transactions['month_lag'].mean()
print('mean month_lag: ', mean_month_lag)
engineered_train['month_lag'] = train[train['target'] > -20].merge(new_merchant_transactions.groupby('card_id')['month_lag'].mean().to_frame(), how='left', on='card_id')['month_lag'].fillna(mean_month_lag)
engineered_train.head()

In [ ]:
engineered_train_outlier['month_lag'] = train[train['target'] < -20].merge(new_merchant_transactions.groupby('card_id')['month_lag'].mean().to_frame(), how='left', on='card_id')['month_lag'].fillna(mean_month_lag)
engineered_train_outlier.head()

In [ ]:
plt.figure()
engineered_train['month_lag'].hist(bins=40)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
engineered_train[engineered_train['month_lag'] == 1.0]['target'].hist(bins=20, ax=ax[0])
engineered_train[engineered_train['month_lag'] == 2.0]['target'].hist(bins=20, ax=ax[1])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(30,3))
for i in range(10):
    title = str(1.0+0.1*i) + '~' + str(1.0+0.1*(i+1))
    engineered_train[(engineered_train['month_lag'] > 1.0+0.1*i) & (engineered_train['month_lag'] < 1.0+0.1*(i+1))]['target'].hist(bins=20, ax=ax[i])
    ax[i].set_title(title)
plt.show()

This tells that although it is mostly normal distribution, but the two extremes encodes some information

# Now historical data. Please give me something useful :(

In [ ]:
historical_transactions.head()

In [ ]:
installments_999 = historical_transactions[historical_transactions['installments'] == 999]

In [ ]:
plt.figure()
installments_999['authorized_flag'].value_counts().sort_index().plot(kind='bar')
plt.show()

Hmmm mystery solved. it seems that 999 is kind of spam that cannot get authorized, but is that useful?

In [ ]:
count_999 = len(historical_transactions[historical_transactions['installments'] == 999])
unauthorized_count = len(historical_transactions[historical_transactions['authorized_flag'] == 'N'])
print('count_999: ', count_999)
print('unauthorized_count: ', unauthorized_count)

In [ ]:
installments_999_unique_cards = installments_999.groupby('card_id')['installments'].mean().to_frame()

In [ ]:
print('total unique 999 cards: ', len(installments_999_unique_cards))
installments_999_unique_cards.head()

In [ ]:
plt.figure()
train.merge(installments_999_unique_cards, how='inner', on='card_id')['target'].hist(bins=40)
plt.show()

hmmmmm what can I say ...

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))
train.merge(historical_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[0])
engineered_train.merge(historical_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[1])
ax[0].set_title('for all the training data')
ax[0].set_xlim([-10, 90])
ax[1].set_title('for training data without outliers')
ax[1].set_xlim([-10, 90])
fig.suptitle('target and mean purchase amount')
plt.show()

In [ ]:
engineered_train['history_purchase_mean'] = engineered_train.merge(historical_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='left', on='card_id')['purchase_amount']

In [ ]:
engineered_test['history_purchase_mean'] = engineered_test.merge(historical_transactions.groupby('card_id')['purchase_amount'].mean().to_frame(), how='left', on='card_id')['purchase_amount']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))
train.merge(historical_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[0])
engineered_train.merge(historical_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='inner', on='card_id').plot.scatter(y='target', x='purchase_amount', ax=ax[1])
ax[0].set_title('for all the training data')
ax[0].set_xlim([-800, 500])
ax[1].set_title('for training data without outliers')
ax[1].set_xlim([-800, 500])
fig.suptitle('target and mean purchase amount')
plt.show()

In [ ]:
engineered_train['history_purchase_sum'] = engineered_train.merge(historical_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='left', on='card_id')['purchase_amount'].fillna(0)

In [ ]:
engineered_test['history_purchase_sum'] = engineered_test.merge(historical_transactions.groupby('card_id')['purchase_amount'].sum().to_frame(), how='left', on='card_id')['purchase_amount'].fillna(0)

# Do some preprocessing to make life easier

In [ ]:
min_first_active_month = min(engineered_train['first_active_month_timestamp'].min(), engineered_test['first_active_month_timestamp'].min())
max_first_active_month = max(engineered_train['first_active_month_timestamp'].max(), engineered_test['first_active_month_timestamp'].max())
print('first_active_month range: ', min_first_active_month, ' ~ ', max_first_active_month)
first_active_month_range = max_first_active_month - min_first_active_month
print('range: ', first_active_month_range)
plt.figure()
engineered_test['first_active_month_timestamp'].hist()
plt.show()

In [ ]:
def normalize_timestamp(timestamp):
    return (timestamp - min_first_active_month) / first_active_month_range

In [ ]:
engineered_train['normalized_timestamp'] = engineered_train['first_active_month_timestamp'].apply(normalize_timestamp)

In [ ]:
engineered_test['normalized_timestamp'] = engineered_test['first_active_month_timestamp'].apply(normalize_timestamp)

In [ ]:
def normalize_target(target):
    return (target + 20) / 40

In [ ]:
engineered_train['normalized_target'] = engineered_train['target'].fillna(0).apply(normalize_target)

In [ ]:
plt.figure()
engineered_train['normalized_target'].hist(bins=40)
plt.show()

In [ ]:
min_history_purchase_sum = min(engineered_train['history_purchase_sum'].min(), engineered_test['history_purchase_sum'].min())
max_history_purchase_sum = max(engineered_train['history_purchase_sum'].max(), engineered_test['history_purchase_sum'].max())
history_purchase_sum_na = engineered_train['history_purchase_sum'].isna().count()
history_purchase_sum_not_na = len(engineered_train) - history_purchase_sum_na
print('history_purchase_sum_na: ', history_purchase_sum_na)
print('history_purchase_sum_not_na: ', history_purchase_sum_not_na)
print('history_purchase_sum range: ', min_history_purchase_sum, ' ~ ', max_history_purchase_sum)
history_purchase_sum_range = max_history_purchase_sum - min_history_purchase_sum
print('range: ', history_purchase_sum_range)
plt.figure()
engineered_train['history_purchase_sum'].hist()
plt.show()
plt.figure()
engineered_train[(engineered_train['history_purchase_sum'].fillna(0) < 200) & (engineered_train['history_purchase_sum'].fillna(0) > -500)]['history_purchase_sum'].hist(bins=40)
plt.show()

In [ ]:
def normalize_history_purchase_sum(history_purchase_sum):
    if history_purchase_sum < -500.0:
        return 0
    elif history_purchase_sum > 200.0:
        return 1
    else:
        return (history_purchase_sum + 500.0) / 700.0

In [ ]:
engineered_train['normalized_history_purchase_sum'] = engineered_train['history_purchase_sum'].fillna(0).apply(normalize_history_purchase_sum)

In [ ]:
engineered_test['normalized_history_purchase_sum'] = engineered_test['history_purchase_sum'].fillna(0).apply(normalize_history_purchase_sum)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,5))
engineered_train['normalized_history_purchase_sum'].hist(ax=ax[0], bins=40)
engineered_test['normalized_history_purchase_sum'].hist(ax=ax[1], bins=40)
plt.show()

In [ ]:
min_history_purchase_mean = min(engineered_train['history_purchase_mean'].min(), engineered_test['history_purchase_mean'].min())
max_history_purchase_mean = max(engineered_train['history_purchase_mean'].max(), engineered_test['history_purchase_mean'].max())
print('max_history_purchase_mean range: ', min_history_purchase_mean, ' ~ ', max_history_purchase_mean)
history_purchase_mean_range = max_history_purchase_mean - min_history_purchase_mean
print('range: ', history_purchase_mean_range)
plt.figure()
engineered_train[(engineered_train['history_purchase_mean'].fillna(0) < 2) & (engineered_train['history_purchase_mean'].fillna(0) > -1)]['history_purchase_mean'].hist(bins=40)
plt.show()

In [ ]:
def normalize_history_purchase_mean(history_purchase_mean):
    if history_purchase_mean < -1:
        return 0
    elif history_purchase_mean > 2:
        return 1
    return (history_purchase_mean + 1) / 3

In [ ]:
engineered_train['normalized_history_purchase_mean'] = engineered_train['history_purchase_mean'].fillna(0).apply(normalize_history_purchase_mean)

In [ ]:
engineered_test['normalized_history_purchase_mean'] = engineered_test['history_purchase_mean'].fillna(0).apply(normalize_history_purchase_mean)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,5))
engineered_train['normalized_history_purchase_mean'].hist(ax=ax[0], bins=40)
engineered_test['normalized_history_purchase_mean'].hist(ax=ax[1], bins=40)
plt.show()

# This section outputs some files that can be used instead of the original files

In [ ]:
historical_transactions[:100].to_csv('light_historical_transactions.csv')

In [ ]:
engineered_train.head()

In [ ]:
engineered_test.head()

In [ ]:
engineered_train[:100].to_csv('light_engineered_train.csv')

In [ ]:
engineered_test[:100].to_csv('light_engineered_test.csv')

In [ ]:
engineered_train.to_csv('engineered_train.csv')

In [ ]:
engineered_test.to_csv('engineered_test.csv')

# Do some sanity check to see if it make sense at all

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.normalization import BatchNormalization

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='sgd', loss='mean_squared_error')

In [ ]:
tags = ['normalized_timestamp', 'normalized_history_purchase_sum', 'normalized_history_purchase_mean']

In [ ]:
training_set = engineered_train[tags].values
training_set

In [ ]:
training_target = engineered_train['target'].values
training_target

In [ ]:
testing_set = engineered_test[tags].values
testing_set

In [ ]:
history = model.fit(x=training_set, y=training_target, epochs=50)

In [ ]:
pred = model.predict(x=testing_set, verbose=1)

In [ ]:
pred.flatten()

In [ ]:
print('prediction length: ', len(pred.flatten().tolist()))
print('engineered test length: ', len(engineered_test['card_id']))

In [ ]:
output_result = pd.DataFrame({'card_id': engineered_test['card_id'].values.tolist(), 'target': pred.flatten().tolist()})

In [ ]:
output_result.head()

In [ ]:
def denormalize_target(target):
    return target * 40 - 20

In [ ]:
output_result['target'] = engineered_train['target'].fillna(0.5).apply(normalize_target)

In [ ]:
output_result.fillna(0.0)

In [ ]:
output_result.fillna(0.0).to_csv('output_result.csv', index=False)